# Compute the Median

In [1]:
%matplotlib notebook
%pylab
import time, multiprocessing
import pandas
import random

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
array_size =2**12
A = random.sample(range(1, array_size*2), array_size)

## Meadian Algorithm
##### Source : http://www.new-npac.org/users/fox/pdftotal/sccs-0743.pdf

In [3]:
def MoM(L, j, q):
    if len(L) < 10:
        L.sort()
        q.put(L[j])
        return L[j]
    S = []
    lIndex = 0
    while lIndex+5 < len(L)-1:
        S.append(L[lIndex:lIndex+5])
        lIndex += 5
    S.append(L[lIndex:])
    Meds = []
    for subList in S:
        Meds.append(MoM(subList, int((len(subList)-1)/2) , q))
    med = MoM(Meds, int((len(Meds)-1)/2), q)
    q.put(med)
    L1 = []
    L2 = []
    L3 = []
    for i in L:
        if i < med:
            L1.append(i)
        elif i > med:
            L3.append(i)
        else:
            L2.append(i)
    if j < len(L1):
        return MoM(L1, j, q)
    elif j < len(L2) + len(L1):
        return L2[0]
    else:
        return MoM(L3, j-len(L1)-len(L2), q)

In [4]:
def compute_median(np, rank):
    N = len(A)
    jobs = []
    medians = []
    q = multiprocessing.Queue()
    for i in range(np):
        i1 = int(i*N/np)
        i2 = int((i+1)*N/np)
        job = multiprocessing.Process(target=MoM, args=[A[i1:i2], int((len(A[i1:i2])-1)/2), q])
        jobs.append(job)
        
    for job in jobs:
        job.start()
    
    for job in jobs:
        job.join()
        
    while not q.empty(): 
        medians.append(q.get())
    print(medians[len(medians)-1])

In [5]:
%time
np = 1
rank = int((len(A)-1)/2)
compute_median(np,rank)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
4044


In [6]:
%time
B = A
B.sort()
B[int((len(A)-1)/2)]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


4044

Median of median (MoM) algorithm outputs the correct answer when single processor is used. This algorithm is implemented using the paper(cited above). However, on multiple CPU's, the MoM returns closer but not accurate result. No load balancing was implemented which in this algorithm is necessary. <br/>

The timing results reported above are from the compute_median algorithm and from built-in sort function. The simple sort function (in theory) takes nlogn time. This algorithms will take (n/5)log(n/5) time to find the nth smallest element. 